In [1]:
import os, sys

from joblib import load
from math import exp,log
from rdkit import Chem, DataStructs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import numpy
import numpy as np
import pickle

import pandas as pd

from rdkit.Chem import Draw
from rdkit.Chem import AllChem, PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import SaltRemover

from rdkit.Chem import PandasTools
from rdkit.Chem import QED
from rdkit.Chem import Fragments
from rdkit import ML
import argparse
import psycopg2

from sshtunnel import SSHTunnelForwarder

import openpyxl

from datetime import datetime
from anytree import Node, RenderTree 
from anytree import Node, RenderTree, AsciiStyle, ZigZagGroupIter, findall,findall_by_attr
import multiprocessing as mp
from functools import partial
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

/home/finith/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# UTIL FUNCTIONS

In [2]:
def convert_SMILES_to_SMARTS(x):
    mol = Chem.MolFromSmiles(x)
    smarts = Chem.MolToSmarts(mol)
    return smarts

def convert_SMILES_to_MOL(x):
    mol = Chem.MolFromSmiles(x)
    remover = SaltRemover.SaltRemover(defnData="[Cl,Br,HCl]")
    mol = remover.StripMol(mol1)
    return mol

def convert_SMARTS_to_SMILES(x):
    smarts = overall_df.loc[overall_df['SMARTS']==x, 'SMILES']
    return smarts

def node_search(level1, level0): 
        try:
            mol = Chem.MolFromSmiles(level1)
            patt = Chem.MolFromSmarts(level0)
            hit_ats = int(mol.HasSubstructMatch(patt))

            if hit_ats > 0:
                    return level0
            else:
                    return 0
                
        except AttributeError :
            #print('syntax error for smiles')
            return 0
                  
        
def rdkit_tanimoto(mol1, mol2):
    try:
        mol1 =  rdkit.Chem.MolFromSmiles(mol1)
        mol2 =  rdkit.Chem.MolFromSmiles(mol2)
        fp1 = rdkit.Chem.RDKFingerprint(mol1)
        fp2 = rdkit.Chem.RDKFingerprint(mol2)

        tanimoto_score = rdkit.DataStructs.TanimotoSimilarity(fp1,fp2)
        return tanimoto_score
    except:
         return None
        
from ast import literal_eval
def extended(x,y):
    overall = []
    overall.extend(x)
    overall.append(y)
    return overall

from ast import literal_eval
def literal(x):
    #try:
    return literal_eval(x)
    #except:
    #    return None


# LOAD DATA

In [3]:
# file structure 

# level 0 and level 1
start0 = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/D1/D1_Node_0a_mono_unsub_NH_to_N_nodup.csv')
start1 = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/D1/D1_Node_0a_1a_mono_sub_NH_to_N_nodup.csv')
start2 = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/D1/D1_Node_0a_1b_bicyclic_unsub_NH_to_N_nodup.csv')
#start1a = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/D1/D1_Node_0a_1a_2a_bicyclic_sub_NH_to_N_nodup.csv')
start1 = start1.append(start2)

# top right
start43 = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/D2/D2_Node_0a_SMARTS_FGs_2_left_c.txt',sep='/t')
start43_kids = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/D2/D2_Node_0a_1a_FGs_SMARTS_FGs_c.csv',sep=',')

# middle block
start27 = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/3_Ar_SMARTS_SMILES_linker_Ar/1_MM/D1_Node_0a_1c_MM_CN_all_Xe_enumeration__SMARTS_linkers_NH_to_N_nodup.csv')
start27_kids = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/3_Ar_SMARTS_SMILES_linker_Ar/1_MM/D1_Node_0a_1c_2c_MM_CN_all_Xe_enumeration_SMILES_SMARTS_nodup_NH_to_N_nodup.csv')

# wildcards 73  
wild_73 = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/wildcards_mono_connodup_730.csv')
wild_73k = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/wildcards_bicyclic_connodup_73191.csv')

# wildcards wc_sub_aro - mono
wild_7 = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/7_wildcards_final_files/WC_Sub_Aro/wildcard_sub_mono_Cl_H_aro.csv')
wild_7['SMARTS_WC_sub_aro'] = wild_7['SMARTS_WC_sub_aro'].apply(lambda x : literal(x))
wild_7['Combined'] = wild_7.apply(lambda x: extended(x['SMARTS_WC_sub_aro'], x['SMARTS_WC_sub']), axis=1)

#
wild_7_bi = pd.read_csv('11182021_SMILES_SMARTS_Hierarchy/7_wildcards_final_files/WC_Sub_Aro/wildcard_sub_bicyclic_Cl_H_aro_SMILES_H.csv')
wild_7_bi['SMARTS_WC_sub_aro'] = wild_7_bi['SMARTS_WC_sub_aro'].apply(lambda x : literal(x))
wild_7_bi['SMILES_H'] = wild_7_bi['SMILES_H'].apply(lambda x : literal(x))
def return_val(x):
    val = x[0]
    return x[0]

wild_7_bi['SMILES_H']= wild_7_bi['SMILES_H'].apply(return_val)
wild_7_bi['Combined'] = wild_7_bi.apply(lambda x: extended(x['SMARTS_WC_sub_aro'], x['SMARTS_WC_sub']), axis=1)
# smiles cl match
# ask for aro and sub


print('DONE HERE')
# create overall_df for smarts to smiles conversion
overall_df = pd.concat([start0, start1, start27, start27_kids, start43, start43_kids])
overall_df = overall_df['SMILES'].fillna(overall_df['SMARTS'])

print('DONE HERE')

/home/finith/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':


DONE HERE
DONE HERE


# TREE SEARCH

In [4]:
def tree_building_1():

    list1 = start0['SMARTS'].tolist()
    start_0 = Node('START')
    
    for i in range(len(list1)):
        print(i)
        a = Node(list1[i], parent=start_0)

        start_smiles = start1['SMILES'].tolist()
        pool = mp.Pool(processes = (mp.cpu_count()-1))
        results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
        pool.close()
        pool.join()

        start1['CHECK'] = results
        # select smiles that have matches to list1[i]
        temp = start1.loc[start1['CHECK']!=0, 'SMARTS' ]
        temp = temp.drop_duplicates()
        temp = temp.tolist()

        keep_temp = []
        if temp:
            for j in range(len(temp)):
                b = Node(temp[j], parent=a)


    return start_0

In [5]:
# creates tree 
# 43 -> 114k

def node_string_search(level1, level0): 
    return level0 in level1 

def tree_building_2():
    list1 = start43['SMARTS'].tolist()
    start_043 = Node('START')
    superset = []
    for i in range(len(list1)):
        
        print(i)
        a = Node(list1[i], parent=start_043)

        start_smiles = start43_kids['SMARTS'].tolist()
        pool = mp.Pool(processes = (mp.cpu_count()-1))
        results = pool.map(partial(node_string_search, level0=list1[i]), start_smiles)
        pool.close()
        pool.join()

        start43_kids['CHECK'] = results
        temp = start43_kids.loc[start43_kids['CHECK']==True, 'SMARTS' ]
        temp = temp.drop_duplicates()
        temp = temp.tolist()
        
        for j in temp:
            if j not in superset:
                superset.append(j)
            else:
                temp.remove(j)

        if temp:
            for j in range(len(temp)):
                b = Node(temp[j], parent=a)

        
    return start_043

# GENERATING FILES

In [6]:
# 23000 x 53000
def tree_building_middleblock_1():
    
    list1 = start27['SMARTS'].tolist()
    csv_list = []
    start_027 = Node('START')
    superset = []
    
    for i in range(len(list1)):
        
        print(i)
        a = Node(list1[i], parent=start_027)

        start_smiles = start27_kids['SMILES'].tolist()
        pool = mp.Pool(processes = (mp.cpu_count()-1))
        results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
        pool.close()
        pool.join()

        start27_kids['CHECK'] = results
        temp = start27_kids.loc[start27_kids['CHECK']!=0, 'SMARTS' ]

        temp = temp.drop_duplicates()
        temp = temp.tolist()
        for j in temp:
            if j not in superset:
                superset.append(j)
            else:
                temp.remove(j)

        if temp:
            csv_list.append([list1[i], temp])
            
    csv_list = pd.DataFrame(csv_list)
    csv_list.to_csv('middle_block_searches.csv')

    return start_027

In [7]:
#matches from 43 parent to 150k

def tree_building_243():
    
    list1 = start43['SMARTS'].tolist()
    start243 = Node('START')
    superset = []
    csv_list = []
    
    for i in range(len(list1)):
        print(i)
        a = Node(list1[i], parent=start243)

        start_smiles = start1['SMILES'].tolist()
        pool = mp.Pool(processes = (mp.cpu_count()-1))
        results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
        pool.close()
        pool.join()

        start1['CHECK'] = results
        temp = start1.loc[start1['CHECK']!=0, 'SMILES' ]
        temp = temp.drop_duplicates()
        temp = temp.tolist()
        
        for j in temp:
            if j not in superset:
                superset.append(j)
            else:
                temp.remove(j)
                
        csv_list.append([list1[i], temp])

        if temp:
            for j in range(len(temp)):
                b = Node(temp[j], parent=a)

        
    df = pd.DataFrame(csv_list)
    df.to_csv('43_to_150k_searches.csv')
    return start243

# find children from 150k and 114k that have 44 in common
def tree_building_243_part2(node243):
    
    # find which of the 43 had matches
    check_for_clustering = []
    children_for_checking = []
    for i in list(node243.children):
        if len(list(i.children)) > 0: 
            check_for_clustering.append(i.name)
            names = []
            for j in i.children:
                names.append(j.name)
            children_for_checking.append(names)
            
    corresponding_values = pd.DataFrame()
    corresponding_values['44'] = check_for_clustering
    corresponding_values['150k'] = children_for_checking
            
            
    # select the orignal children of those 43 (43->114k) -OLD 
    original_children = []
    for i in node2.children:
        if (i.name in check_for_clustering) and (len(list(i.children)) > 0):
            selected_kids = []
            for j in list(i.children):
                selected_kids.append(j.name)
            original_children.append([i.name,selected_kids])
            
    original_children = pd.DataFrame(original_children)
    original_children.columns =['44', '114k']
            
    #44
    #corresponding 150 for 44 
    #corresponding 114 for 44
    corresponding_values = pd.merge(corresponding_values, original_children, on='44')
    
    # check for duplicates
    def check_dupes(x):
        return list(set(x))
    
    corresponding_values['114k'] = corresponding_values['114k'].apply(lambda x: check_dupes(x))
    corresponding_values['150k'] = corresponding_values['150k'].apply(lambda x: check_dupes(x))
     
    return corresponding_values


# match selected 114k -> selected 150k
def tree_building_part3(node):
    list_main = node['114k'].tolist()
    search_terms = node['150k'].tolist()
    csv_list = []
    
    # for list1 in the list of 114, find corresponding matches of 114 to corresponding 150k
    print(len(list_main),'first')
    
    for i in range(len(list_main)):
        print(len(list_main[i]), 'second')
                
        searched = pd.DataFrame(search_terms[i])
        start_smiles = search_terms[i]
    
        for j in list_main[i]:
            pool = mp.Pool(processes = (mp.cpu_count()-1))
            results = pool.map(partial(node_search, level0=j), start_smiles)
            pool.close()
            pool.join()
            
            searched['results'] = results
            temp = searched.loc[searched['results']!= 0, 0].tolist()
            csv_list.append([j, temp])
            searched = searched[searched['results']==0]
            start_smiles = searched[0].tolist()

            
    csv_list = pd.DataFrame(csv_list)
    csv_list.to_csv('114k_150k.csv')
    
    return csv_list


In [8]:
# 52 -> 730 searches 
def tree_building_52_730():
    
    list1 = wild_73['SMARTS'].tolist()
    csv_list = []
    start_073 = Node('START')
    superset = []
    
    for i in range(len(list1)):
        print(i)
        a = Node(list1[i], parent=start_073)

        start_smiles = start0['SMILES'].tolist()
        pool = mp.Pool(processes = (mp.cpu_count()-1))
        results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
        pool.close()
        pool.join()

        start0['CHECK'] = results
        temp = start0.loc[start0['CHECK']!=0, 'SMILES' ]

        temp = temp.drop_duplicates()
        temp = temp.tolist()
        for j in temp:
            if j not in superset:
                superset.append(j)
            else:
                temp.remove(j)

        if temp:
            for j in temp:
                csv_list.append([j, list1[i]])
            
    csv_list = pd.DataFrame(csv_list)
    csv_list.to_csv('searched_730.csv')

    return start_073

In [9]:
# 1400 -> 73k searches 
def tree_building_1400_73k():
    
    list1 = wild_73k['SMARTS'].tolist()
    csv_list = []
    start_073k = Node('START')
    superset = []
    
    for i in range(len(list1)):
        print(i)
        a = Node(list1[i], parent=start_073k)

        start_smiles = start2['SMILES'].tolist()
        pool = mp.Pool(processes = (mp.cpu_count()-1))
        results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
        pool.close()
        pool.join()

        start2['CHECK'] = results
        temp = start2.loc[start2['CHECK']!=0, 'SMARTS' ]

        temp = temp.drop_duplicates()
        temp = temp.tolist()
        for j in temp:
            if j not in superset:
                superset.append(j)
            else:
                temp.remove(j)

        if temp:
            for j in temp:
                csv_list.append([j, list1[i]])
            
    csv_list = pd.DataFrame(csv_list)
    csv_list.to_csv('searched_730k.csv')

    return start_073

In [10]:
# 150k -> 832 wc_sub and wc_sub_aro 

def tree_building_150k_832():
    
    list1 = wild_7['SMARTS_WC_sub'].tolist()
    list2 = wild_7['SMARTS_WC_sub_aro'].tolist()
    csv_list = []
    
    for i in range(len(list1)):
        print(i)

        start_smiles = start1['SMILES'].tolist()
        pool = mp.Pool(processes = (mp.cpu_count()-1))
        results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
        pool.close()
        pool.join()

        start1['CHECK'] = results
        temp = start1.loc[start1['CHECK']!=0, 'SMILES' ]

        temp = temp.drop_duplicates()
        temp = temp.tolist()

        if temp:
            csv_list.append([temp, list1[i], list2[i]])
            
    csv_list = pd.DataFrame(csv_list)
    csv_list.to_csv('searched_7_aro_sub.csv')

    return csv_list

In [11]:
# 1.4k -> 45k smarts_H 
def tree_building_1400_45k():
    
    list1 = wild_7_bi['SMARTS_H'].tolist()
    list2 = wild_7_bi['Combined'].tolist()
    csv_list = []
    superset = []

    for i in range(len(list1)):
        
            print(i)

            start_smiles = start2['SMILES'].tolist()
            pool = mp.Pool(processes = (mp.cpu_count()-1))
            results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
            pool.close()
            pool.join()

            start2['CHECK'] = results
            temp = start2.loc[start2['CHECK']!=0, 'SMILES' ].tolist()
            print(len(temp))

            if temp :
                # second check
                subset = list2[i]
                for j in range(len(subset)): 

                    start_smiles2 = temp.copy()
                    temp2 = pd.DataFrame(start_smiles2)

                    pool = mp.Pool(processes = (mp.cpu_count()-1))
                    results = pool.map(partial(node_search, level0=subset[j]), start_smiles2)
                    pool.close()
                    pool.join()

                    temp2['CHECK'] = results
                    temp2 = temp2.loc[temp2['CHECK']!=0, 0].tolist()

                    if temp2:
                        print(len(temp2))
                        csv_list.append([list1[i], subset[j], temp2])
                        
                print('YES')
                csv_list.append([list1[i],'None', temp])


    csv_list = pd.DataFrame(csv_list)
    csv_list.to_csv('searched_1k_45k.csv')

    return csv_list

In [27]:
# 1.4k -> 45k smarts_H 
def tree_building_1400_45k_new():
    
    list1 = start2['SMARTS'].tolist()
    list2 = wild_7_bi['SMILES_Cl'].tolist()
    list3 = wild_7_bi['Combined'].tolist()
    
    csv_list = []
    superset = []

    for i in range(len(list1)):
        
            for j in range(len(list2)):
                
                if node_search(list2[i], list1[i]) != 0:
                    print('YES')
                    start_smiles = list3[i]
                    pool = mp.Pool(processes = (mp.cpu_count()-1))
                    results = pool.map(partial(node_search, level0=list1[i]), start_smiles)
                    pool.close()
                    pool.join()
                    
                    start = pd.DataFrame(list3[i])
                    start['CHECK'] = results
                    temp = start.loc[start['CHECK']!=0, 0 ].tolist()
                    print(len(temp))
                    
                    if temp:
                        csv_list.append(list1[i], temp)

    csv_list = pd.DataFrame(csv_list)
    csv_list.to_csv('searched_1k_45k.csv')

    return csv_list

# RUN TO GENERATE FILES 

In [ ]:
# 39->150k
node1 = tree_building_1()

#43-> 114k
node2 = tree_building_2()

# node1 and node2 need to be created before running these, 150k -> 114k
part1 = tree_building_243()
tree_df = tree_building_243_part2(part1)
clustering_node = tree_building_part3(tree_df)

# 2300 x 53000 middleblock 
tree_23 = tree_building_middleblock_1()

# 52 -> 730 
wild1 = tree_building_52_730()

# 1400 -> 73k 
wild2 = tree_building_1400_73k()

# 150k -> 832 
tree_building_150k_832()

# 1.4k -> 45000
tree_building_1400_45k()